## Assignment : 14

<pre>
1. Preprocess all the Data we have in DonorsChoose <a href='https://drive.google.com/drive/folders/1MIwK7BQMev8f5CbDDVNLPaFGB32pFN60'>Dataset</a> use train.csv
2. Combine 4 essay's into one column named - 'preprocessed_essays'. 
3. After step 2 you have to train 3 types of models as discussed below. 
4. For all the model use <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics'>'auc'</a> as a metric. check <a href='https://datascience.stackexchange.com/a/20192'>this</a> for using auc as a metric 
5. You are free to choose any number of layers/hiddden units but you have to use same type of architectures shown below. 
6. You can use any one of the optimizers and choice of Learning rate and momentum, resources: <a href='http://cs231n.github.io/neural-networks-3/'>cs231n class notes</a>, <a href='https://www.youtube.com/watch?v=hd_KFJ5ktUc'>cs231n class video</a>. 
7. For all the model's use <a href='https://www.youtube.com/watch?v=2U6Jl7oqRkM'>TensorBoard</a> and plot the Metric value and Loss with epoch. While submitting, take a screenshot of plots and include those images in .ipynb notebook and PDF. 
8. Use Categorical Cross Entropy as Loss to minimize.
</pre>

### Model-1

Build and Train deep neural network as shown below

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

- __Input_seq_total_text_data__ --- You have to give Total text data columns. After this use the Embedding layer to get word vectors. Use given predefined glove word vectors, don't train any word vectors. After this use LSTM and get the LSTM output and Flatten that output. 
- __Input_school_state__ --- Give 'school_state' column as input to embedding layer and Train the Keras Embedding layer. 
- __Project_grade_category__  --- Give 'project_grade_category' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_categories__ --- Give 'input_clean_categories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_clean_subcategories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_teacher_prefix' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_remaining_teacher_number_of_previously_posted_projects._resource_summary_contains_numerical_digits._price._quantity__ ---concatenate remaining columns and add a Dense layer after that. 



- For LSTM, you can choose your sequence padding methods on your own or you can train your LSTM without padding, there is no restriction on that.

Below is an example of embedding layer for a categorical columns. In below code all are dummy values, we gave only for referance. 

In [ ]:
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
input_layer = Input(shape=(n,))
embedding = Embedding(no_1, no_2, input_length=n)(input_layer)
flatten = Flatten()(embedding)

### 1. Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
### 2. Please go through this link https://keras.io/getting-started/functional-api-guide/ and check the 'Multi-input and multi-output models' then you will get to know how to give multiple inputs. 

### Model-2

Use the same model as above but for 'input_seq_total_text_data' give only some words in the sentance not all the words. Filter the words as below. 

<pre>
1. Train the TF-IDF on the Train data <br>
2. Get the idf value for each word we have in the train data. <br>
3. Remove the low idf value and high idf value words from our data. Do some analysis on the Idf values and based on those values choose the low and high threshold value. Because very frequent words and very very rare words don't give much information. (you can plot a box plots and take only the idf scores within IQR range and corresponding words)<br>
4. Train the LSTM after removing the Low and High idf value words. (In model-1 Train on total data but in Model-2 train on data after removing some words based on IDF values)
</pre>

### Model-3

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png


- __input_seq_total_text_data__: <br>
<pre>
    . Use text column('essay'), and use the Embedding layer to get word vectors. <br>
    . Use given predefined glove word vectors, don't train any word vectors. <br>
    . Use LSTM that is given above, get the LSTM output and Flatten that output. <br>
    . You are free to preprocess the input text as you needed. <br>
</pre>
- __Other_than_text_data__:<br>
<pre>
    . Convert all your Categorical values to onehot coded and then concatenate all these onehot vectors <br>
    . Neumerical values and use <a href='https://keras.io/getting-started/sequential-model-guide/#sequence-classification-with-1d-convolutions'>CNN1D</a> as shown in above figure. <br>
    . You are free to choose all CNN parameters like kernel sizes, stride.<br>
    
</pre>

# Assignment:

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from scipy.sparse import hstack

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/

from nltk.corpus import stopwords
import pickle

from tqdm import tqdm
import os

## Reading data

In [2]:
project_data = pd.read_csv('new_train.csv', nrows=5000)
resource_data = pd.read_csv('resources.csv')

In [3]:
print("Number of data points in train data", project_data.shape)
print('-'*50)
print("The attributes of data :", project_data.columns.values)

Number of data points in train data (5000, 21)
--------------------------------------------------
The attributes of data : ['Unnamed: 0' 'id' 'teacher_id' 'teacher_prefix' 'school_state'
 'project_submitted_datetime' 'project_grade_category' 'project_title'
 'project_essay_1' 'project_essay_2' 'project_essay_3' 'project_essay_4'
 'project_resource_summary' 'teacher_number_of_previously_posted_projects'
 'project_is_approved' 'clean_categories' 'clean_subcategories' 'essay'
 'price' 'quantity' 'is_digit_present']


In [4]:
project_data.head(2)

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_title,project_essay_1,project_essay_2,...,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,quantity,is_digit_present
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,Mrs.,IN,2016-12-05 13:43:57,Grades PreK-2,Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",...,NaN,My students need opportunities to practice beg...,0,0,Literacy_Language,ESL Literacy,My students are English learners that are work...,154.6,23,0
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,Mr.,FL,2016-10-25 09:22:10,Grades 6-8,Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,...,NaN,My students need a projector to help with view...,7,1,History_Civics Health_Sports,Civics_Government TeamSports,Our students arrive to our school eager to lea...,299.0,1,0


In [5]:
print("Number of data points in resource data", resource_data.shape)
print(resource_data.columns.values)
resource_data.head(2)

Number of data points in resource data (1541272, 4)
['id' 'description' 'quantity' 'price']


,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95


In [6]:
x = project_data.drop(['project_is_approved','project_essay_1','project_essay_2','project_essay_3','project_essay_4'], axis=1)
y = project_data['project_is_approved'].values

In [7]:
x.shape

(5000, 16)

In [8]:
y.shape

(5000,)

## Splitting Data

In [9]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.2, random_state=42)    

In [10]:
print(x_train.shape)
print(x_cv.shape)
print(x_test.shape)

(3200, 16)
(800, 16)
(1000, 16)


## Model 1:

In [11]:
from keras.layers import Input, Embedding, LSTM, Dense, Flatten, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from numpy import asarray, zeros
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Conv1D

Using TensorFlow backend.


## Tokenizer

for essay

In [12]:
x_train.head(2)

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,clean_categories,clean_subcategories,essay,price,quantity,is_digit_present
3444,175252,p248801,400f8750770107cdbb9bfbcf852a72ea,Ms.,AZ,2016-10-13 16:19:56,Grades PreK-2,Painting With a Purpose,My students need an art easel and some dot mar...,3,SpecialNeeds Music_Arts,SpecialNeeds VisualArts,Our students come from a variety of different ...,188.24,2,0
2063,136561,p141074,551e854422dccca4adc92781d8bd4e4a,Mrs.,PA,2016-08-03 08:47:25,Grades 3-5,Big Brain Benefits,My students need 21 chess sets to have the too...,1,Literacy_Language Math_Science,Literacy Mathematics,Of all the students in our public schools it m...,9.31,21,1


In [161]:
t = Tokenizer()
t.fit_on_texts(x_train['essay'])
vocab_size   = len(t.word_index) + 1
encoded_train_essay = t.texts_to_sequences(x_train['essay'])

In [162]:
len(encoded_train_essay)

3200

In [163]:
encoded_cv_essay   = t.texts_to_sequences(x_cv['essay'])
encoded_test_essay = t.texts_to_sequences(x_test['essay'])

In [164]:
len(encoded_test_essay)

1000

### Encoding categorical features

In [17]:
x_train.isnull().any()

Unnamed: 0                                      False
id                                              False
teacher_id                                      False
teacher_prefix                                  False
school_state                                    False
project_submitted_datetime                      False
project_grade_category                          False
project_title                                   False
project_resource_summary                        False
teacher_number_of_previously_posted_projects    False
clean_categories                                False
clean_subcategories                             False
essay                                           False
price                                           False
quantity                                        False
is_digit_present                                False
dtype: bool

Here we confirmed before encoding that there are no null values in any column of our dataframe so now we can do the encoding.

In [18]:
# encoding categorical features using label encoder ------> https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

le = LabelEncoder()

#### Encoding School State

In [19]:
x_train['school_state'] = le.fit_transform(x_train['school_state'])
print("Number of Distinct classes in school_state: ",len(le.classes_))
x_cv['school_state']    = le.fit_transform(x_cv['school_state'])
#print(le.classes_)
x_test['school_state']  = le.fit_transform(x_test['school_state'])
#print(le.classes_)

Number of Distinct classes in school_state:  51


#### Encoding Grage category

In [20]:
x_train['project_grade_category'] = le.fit_transform(x_train['project_grade_category'])
print("Number of Distinct classes in project_grade_categories: ",len(le.classes_))
x_cv['project_grade_category']    = le.fit_transform(x_cv['project_grade_category'])
x_test['project_grade_category']  = le.fit_transform(x_test['project_grade_category'])

Number of Distinct classes in project_grade_categories:  4


#### Encoding Clean category

In [21]:
x_train['clean_categories'] = le.fit_transform(x_train['clean_categories'])
print("Number of Distinct classes in clean_categories: ",len(le.classes_))
x_cv['clean_categories']    = le.fit_transform(x_cv['clean_categories'])
x_test['clean_categories']  = le.fit_transform(x_test['clean_categories'])

Number of Distinct classes in clean_categories:  45


#### Encoding Clean sub-category

In [22]:
x_train['clean_subcategories'] = le.fit_transform(x_train['clean_subcategories'])
print("Number of Distinct classes in clean_subcategories: ",len(le.classes_))
x_cv['clean_subcategories']    = le.fit_transform(x_cv['clean_subcategories'])
x_test['clean_subcategories']  = le.fit_transform(x_test['clean_subcategories'])

Number of Distinct classes in clean_subcategories:  219


#### Encoding Teacher prefix

In [23]:
x_train['teacher_prefix'] = le.fit_transform(x_train['teacher_prefix'])
print("Number of Distinct classes in teacher_prefix: ",len(le.classes_))
x_cv['teacher_prefix']    = le.fit_transform(x_cv['teacher_prefix'])
x_test['teacher_prefix']  = le.fit_transform(x_test['teacher_prefix'])

Number of Distinct classes in teacher_prefix:  4


#### Encoding Remaining columns

In [24]:
remaining_cols = ['teacher_number_of_previously_posted_projects','is_digit_present']

In [25]:
ohe = OneHotEncoder(sparse=False) 
ohe.fit(x_train[remaining_cols])

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)

In [26]:
x_train_remaining_cols = ohe.transform(x_train[remaining_cols])
x_cv_remaining_cols    = ohe.transform(x_cv[remaining_cols])
x_test_remaining_cols  = ohe.transform(x_test[remaining_cols])
print(x_train_remaining_cols.shape)
print(x_cv_remaining_cols.shape)
print(x_test_remaining_cols.shape)

(3200, 144)
(800, 144)
(1000, 144)


## Padding

Essay

In [165]:
max_length   = 400
padded_train_essay = pad_sequences(encoded_train_essay, maxlen= max_length, padding='post')
print(padded_train_essay)

[[  18    4   53 ...    0    0    0]
 [   6   37    3 ...    0    0    0]
 [2244  955  987 ...    0    0    0]
 ...
 [   9    4   10 ...    0    0    0]
 [   7 6567    9 ...    0    0    0]
 [  18   21   13 ...    0    0    0]]


In [166]:
max_length   = 400
padded_cv_essay = pad_sequences(encoded_cv_essay, maxlen= max_length, padding='post')
print(padded_cv_essay)

[[ 9  4 10 ...  0  0  0]
 [15 93 82 ...  0  0  0]
 [ 9 21 16 ...  0  0  0]
 ...
 [15 79  5 ...  0  0  0]
 [ 9  4 19 ...  0  0  0]
 [ 9  4 10 ...  0  0  0]]


In [167]:
max_length   = 400
padded_test_essay = pad_sequences(encoded_test_essay, maxlen= max_length, padding='post')
print(padded_test_essay)

[[9546 9205  330 ...    0    0    0]
 [   9    4   10 ...    0    0    0]
 [   9    4   10 ...    0    0    0]
 ...
 [ 228   16   95 ...    0    0    0]
 [ 367  278   14 ...    0    0    0]
 [   9    4   10 ...   12   26   46]]


## Load entire glove embedding

In [30]:
embedding_index = dict()
f = open('glove.6B.50d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word   = values[0]
    coefs  = asarray(values[1:], dtype='float32') 
    embedding_index[word] = coefs
f.close()
print('Loaded %s word vectors' % len(embedding_index))

Loaded 400000 word vectors


In [31]:
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items() :
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Here we are going to use Functional API for creating our architecture.

In [172]:
ip_seq_total_text = Input(shape = (400,), name = 'ip_total_text')
x = Embedding(output_dim=128, input_dim=400000, input_length=400)(ip_seq_total_text)
x = LSTM(32, return_sequences=True)(x)
x    = Flatten()(x)

ip_school_state = Input(shape=(1,), name= 'ip_school_state')
y  = Embedding(output_dim=1, input_dim=51, input_length=1)(ip_school_state)
y  = Flatten()(y)

ip_grade_category = Input(shape=(1,), name = 'ip_grade_category')
z = Embedding(output_dim=1, input_dim=4, input_length=1)(ip_grade_category)
z = Flatten()(z)

ip_clean_category = Input(shape=(1,), name = 'ip_clean_category')
a = Embedding(output_dim=1, input_dim=45, input_length=1)(ip_clean_category)
a = Flatten()(a)

ip_clean_subcategory= Input(shape=(1,), name ='ip_clean_subcategory')
b = Embedding(output_dim=1, input_dim=219, input_length=1)(ip_clean_subcategory)
b = Flatten()(b)

ip_teacher_prefix= Input(shape=(1,), name ='ip_teacher_prefix')
c = Embedding(output_dim=1, input_dim=4, input_length=1)(ip_teacher_prefix)
c = Flatten()(c)

ip_combined_columns = Input(shape=(144,), name= 'ip_combined_columns')
d = Dense(64, activation='relu')(ip_combined_columns)
#"""
model = concatenate([x,y,z,a,b,c,d])
model = Dense(64, activation='relu')(model)
model = Dropout(0.5)(model)

model = Dense(32, activation='relu')(model)
model = Dropout(0.25)(model)

model = Dense(16, activation='relu')(model)
output_layer = Dense(2, activation="sigmoid")(model)

model = Model(inputs = [ip_seq_total_text,ip_school_state,ip_grade_category,ip_clean_category,ip_clean_subcategory, ip_teacher_prefix, ip_combined_columns], outputs= output_layer)
# Compiling the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip_total_text (InputLayer)      (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_43 (Embedding)        (None, 400, 128)     51200000    ip_total_text[0][0]              
__________________________________________________________________________________________________
ip_school_state (InputLayer)    (None, 1)            0                                            
__________________________________________________________________________________________________
ip_grade_category (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
ip_clean_c

In [173]:
metrics = Metrics()
plot = model.fit([padded_train_essay, x_train['school_state'], x_train['project_grade_category'], x_train['clean_categories'], x_train['clean_subcategories'], x_train['teacher_prefix'], x_train_remaining_cols],y_train, epochs=10, batch_size=128, verbose=2, validation_data=([padded_cv_essay, x_cv['school_state'], x_cv['project_grade_category'], x_cv['clean_categories'], x_cv['clean_subcategories'], x_cv['teacher_prefix'], x_cv_remaining_cols], y_cv))

Train on 3200 samples, validate on 800 samples
Epoch 1/10
 - 64s - loss: 0.5215 - acc: 0.8253 - val_loss: 0.4300 - val_acc: 0.8575
Epoch 2/10
 - 34s - loss: 0.4452 - acc: 0.8472 - val_loss: 0.4080 - val_acc: 0.8575
Epoch 3/10
 - 36s - loss: 0.3810 - acc: 0.8478 - val_loss: 0.4104 - val_acc: 0.8575
Epoch 4/10
 - 36s - loss: 0.2440 - acc: 0.8847 - val_loss: 0.6198 - val_acc: 0.7925
Epoch 5/10
 - 36s - loss: 0.1130 - acc: 0.9616 - val_loss: 0.9207 - val_acc: 0.8263
Epoch 6/10
 - 38s - loss: 0.0386 - acc: 0.9888 - val_loss: 1.3011 - val_acc: 0.7987
Epoch 7/10
 - 38s - loss: 0.0288 - acc: 0.9906 - val_loss: 1.4876 - val_acc: 0.8550
Epoch 8/10
 - 38s - loss: 0.0209 - acc: 0.9944 - val_loss: 1.3930 - val_acc: 0.8050
Epoch 9/10
 - 37s - loss: 0.0050 - acc: 0.9988 - val_loss: 1.9442 - val_acc: 0.8025
Epoch 10/10
 - 37s - loss: 0.0070 - acc: 0.9994 - val_loss: 1.7189 - val_acc: 0.8237


In [143]:
"""
from keras.utils import plot_model
plot_model(model, to_file='model.png')
"""

# Model 2:

Tfidf Vectorization

In [103]:
x_train.shape

(3200, 16)

In [82]:
tf_idf = TfidfVectorizer( use_idf=True) # min_df=10
tf_idf.fit(x_train['essay'])
print("some sample features",tf_idf.get_feature_names()[0:50]) 
print('='*50)

x_train_tfidf = tf_idf.transform(x_train['essay']) 
x_cv_tfidf    = tf_idf.transform(x_cv['essay'])
x_test_tfidf  = tf_idf.transform(x_test['essay'])

print("After featurization\n")
print(x_train_tfidf.shape, y_train.shape)
print(x_cv_tfidf.shape,    y_cv.shape)
print(x_test_tfidf.shape,  y_test.shape) 

some sample features ['00', '000', '01', '02', '047', '05', '0nly', '10', '100', '1000', '100m', '100s', '100th', '101', '102', '105', '107', '1084', '10pm', '10th', '11', '110', '1100', '116', '11th', '11x17', '12', '120', '12000', '120s', '125', '127', '12th', '13', '130', '1300', '135', '14', '140', '15', '150', '1500', '16', '160', '1600', '162nd', '16th', '17', '170', '175']
After featurization

(3200, 15277) (3200,)
(800, 15277) (800,)
(1000, 15277) (1000,)


In [83]:
# we are converting a dictionary with word as a key, and the idf as a value 
dictionary_train = dict(zip(tf_idf.get_feature_names(), list(tf_idf.idf_))) 

In [84]:
len(dictionary_train)

15277

In [85]:
filtered_dict = dict()
for (key, value) in dictionary_train.items():
    # Check if key is even then add pair to new dictionary
    if value < 3:
        continue
    elif value > 6:
        continue
    else :
        filtered_dict[key] = value
        # newDict[key] = value
print('Filtered Dictionary : ')
print(filtered_dict)

Filtered Dictionary : 


{'000': 5.703922709983389, '10': 4.466048353981772, '100': 3.664046768509744, '11': 5.5158704784804495, '12': 4.689191905295981, '15': 5.139392907245537, '16': 5.8523427151016625, '17': 5.893164709621917, '18': 5.5158704784804495, '1st': 5.028167272135313, '20': 4.42682764082849, '2016': 5.407656893840216, '2017': 5.813122001948381, '21': 5.5158704784804495, '21st': 4.034265937556234, '22': 5.433632380243477, '23': 5.980176086611547, '24': 5.100926626417741, '25': 4.976873977747762, '26': 5.8523427151016625, '28': 5.6054826371701365, '2nd': 4.49650756146648, '30': 4.66449929270561, '3d': 5.382339085855927, '3rd': 4.407779445857797, '40': 5.0107755294234435, '4th': 4.446245726685592, '50': 4.822723297920504, '500': 5.6054826371701365, '5th': 3.92372406315641, '60': 4.676769385297424, '600': 5.8523427151016625, '6th': 4.605310421315279, '70': 5.487699601513753, '75': 5.433632380243477, '7th': 5.100926626417741, '80': 5.082234493405589, '85': 5.544858015353702, '8th': 4.517341648369323, '

In [86]:
len(filtered_dict)

2136

In [89]:
# putting these words into a set

words = []

for (key, value) in filtered_dict.items():
    words.append(key)

In [90]:
len(words)

2136

In [129]:
tf_idf.fit(words)

x_train_tfidf = tf_idf.transform(x_train['essay']) 
x_cv_tfidf    = tf_idf.transform(x_cv['essay'])
x_test_tfidf  = tf_idf.transform(x_test['essay'])

print("After featurization\n")
print(x_train_tfidf.shape, y_train.shape)
print(x_cv_tfidf.shape,    y_cv.shape)
print(x_test_tfidf.shape,  y_test.shape) 

After featurization

(3200, 2136) (3200,)
(800, 2136) (800,)
(1000, 2136) (1000,)


## Tokenizing

In [130]:
t = Tokenizer()
t.fit_on_texts(words)
vocab_size   = len(t.word_index) + 1
encoded_train_essay = t.texts_to_sequences(x_train['essay'])

In [131]:
len(encoded_train_essay)

3200

In [132]:
encoded_cv_essay   = t.texts_to_sequences(x_cv['essay'])
encoded_test_essay = t.texts_to_sequences(x_test['essay'])

In [92]:
"""
encoded_cv_essay   = t.texts_to_sequences(words)
encoded_test_essay = t.texts_to_sequences(words)
"""

In [133]:
len(encoded_test_essay)

1000

## Padding

In [147]:
max_length   = 118
padded_train_essay = pad_sequences(encoded_train_essay, maxlen= max_length, padding='post')
print(padded_train_essay)

[[ 473 1201  562 ...    0    0    0]
 [1549 1688 1199 ...    0    0    0]
 [1816  950 1591 ...    0    0    0]
 ...
 [1903 1163  923 ...    0    0    0]
 [2003 2005  812 ...    0    0    0]
 [1682 2093  341 ...    0    0    0]]


In [146]:
max_length   = 118
padded_cv_essay = pad_sequences(encoded_cv_essay, maxlen= max_length, padding='post')
print(padded_cv_essay)

[[2005 1278  319 ...    0    0    0]
 [2005  342  855 ...    0    0    0]
 [1580 1688 2110 ...    0    0    0]
 ...
 [1722 1731  637 ...    0    0    0]
 [1490   44  331 ...    0    0    0]
 [1953  676  636 ...    0    0    0]]


In [145]:
max_length   = 118
padded_test_essay = pad_sequences(encoded_test_essay, maxlen= max_length, padding='post')
print(padded_test_essay)

[[ 631 1236 1807 ...    0    0    0]
 [ 536  284  611 ...    0    0    0]
 [1253  823 1036 ...    0    0    0]
 ...
 [1662 1012  200 ...    0    0    0]
 [1762 1855  734 ...    0    0    0]
 [ 258 1013  288 ... 1375   65 1407]]


In [149]:
ip_seq_total_text = Input(shape = (118,), name = 'ip_total_text')
x = Embedding(output_dim=128, input_dim=400000, input_length=118)(ip_seq_total_text)
x = LSTM(32, return_sequences=True)(x)
x    = Flatten()(x)

ip_school_state = Input(shape=(1,), name= 'ip_school_state')
y  = Embedding(output_dim=1, input_dim=51, input_length=1)(ip_school_state)
y  = Flatten()(y)

ip_grade_category = Input(shape=(1,), name = 'ip_grade_category')
z = Embedding(output_dim=1, input_dim=4, input_length=1)(ip_grade_category)
z = Flatten()(z)

ip_clean_category = Input(shape=(1,), name = 'ip_clean_category')
a = Embedding(output_dim=1, input_dim=45, input_length=1)(ip_clean_category)
a = Flatten()(a)

ip_clean_subcategory= Input(shape=(1,), name ='ip_clean_subcategory')
b = Embedding(output_dim=1, input_dim=219, input_length=1)(ip_clean_subcategory)
b = Flatten()(b)

ip_teacher_prefix= Input(shape=(1,), name ='ip_teacher_prefix')
c = Embedding(output_dim=1, input_dim=4, input_length=1)(ip_teacher_prefix)
c = Flatten()(c)

ip_combined_columns = Input(shape=(144,), name= 'ip_combined_columns')
d = Dense(64, activation='relu')(ip_combined_columns)
#"""
model = concatenate([x,y,z,a,b,c,d])
model = Dense(64, activation='relu')(model)
model = Dropout(0.5)(model)

model = Dense(32, activation='relu')(model)
model = Dropout(0.25)(model)

model = Dense(16, activation='relu')(model)
output_layer = Dense(2, activation="sigmoid")(model)

model = Model(inputs = [ip_seq_total_text,ip_school_state,ip_grade_category,ip_clean_category,ip_clean_subcategory, ip_teacher_prefix, ip_combined_columns], outputs= output_layer)
# Compiling the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip_total_text (InputLayer)      (None, 118)          0                                            
__________________________________________________________________________________________________
embedding_25 (Embedding)        (None, 118, 128)     51200000    ip_total_text[0][0]              
__________________________________________________________________________________________________
ip_school_state (InputLayer)    (None, 1)            0                                            
__________________________________________________________________________________________________
ip_grade_category (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
ip_clean_c

In [156]:
plot = model.fit([padded_train_essay, x_train['school_state'], x_train['project_grade_category'], x_train['clean_categories'], x_train['clean_subcategories'], x_train['teacher_prefix'], x_train_remaining_cols],y_train, epochs=10, batch_size=128, verbose=2, validation_data=([padded_cv_essay, x_cv['school_state'], x_cv['project_grade_category'], x_cv['clean_categories'], x_cv['clean_subcategories'], x_cv['teacher_prefix'], x_cv_remaining_cols], y_cv))

Train on 3200 samples, validate on 800 samples
Epoch 1/10
 - 24s - loss: 5.1854e-05 - acc: 1.0000 - val_loss: 2.8190 - val_acc: 0.7863
Epoch 2/10
 - 24s - loss: 3.0572e-04 - acc: 0.9997 - val_loss: 2.8325 - val_acc: 0.7863
Epoch 3/10
 - 23s - loss: 3.4679e-05 - acc: 1.0000 - val_loss: 2.8438 - val_acc: 0.7863
Epoch 4/10
 - 24s - loss: 4.2036e-05 - acc: 1.0000 - val_loss: 2.8557 - val_acc: 0.7850
Epoch 5/10
 - 26s - loss: 1.6915e-04 - acc: 1.0000 - val_loss: 2.8927 - val_acc: 0.7825
Epoch 6/10
 - 30s - loss: 5.5480e-05 - acc: 1.0000 - val_loss: 2.9049 - val_acc: 0.7837
Epoch 7/10
 - 27s - loss: 5.9717e-05 - acc: 1.0000 - val_loss: 2.9107 - val_acc: 0.7837
Epoch 8/10
 - 28s - loss: 4.4371e-05 - acc: 1.0000 - val_loss: 2.9051 - val_acc: 0.7837
Epoch 9/10
 - 29s - loss: 9.2774e-05 - acc: 1.0000 - val_loss: 2.9049 - val_acc: 0.7863
Epoch 10/10
 - 28s - loss: 1.8611e-04 - acc: 1.0000 - val_loss: 2.9048 - val_acc: 0.7863


# Model 3:

## Encoding categorical features using OneHot encoder

In [39]:
list_of_categorical_features = ['school_state', 'teacher_prefix', 'project_grade_category', 'teacher_number_of_previously_posted_projects', 'clean_categories', 'clean_subcategories',  'is_digit_present']

In [40]:
len(list_of_categorical_features)

7

In [49]:
# encoding categorical features  --------->    https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

#ohe = CountVectorizer()
ohe = OneHotEncoder(sparse=False) 
ohe.fit(x_train[list_of_categorical_features])

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)

In [50]:
x_train_ohe = ohe.transform(x_train[list_of_categorical_features])
x_cv_ohe    = ohe.transform(x_cv[list_of_categorical_features])
x_test_ohe  = ohe.transform(x_test[list_of_categorical_features])
print(x_train_ohe.shape)
print(x_cv_ohe.shape)
print(x_test_ohe.shape)

(3200, 467)
(800, 467)
(1000, 467)


Reshaping the train, cv and test data

In [89]:
x_train_ohe = x_train_ohe.reshape(-1, 467, 1)
x_cv_ohe    = x_cv_ohe.reshape(-1, 467, 1)
x_test_ohe  = x_test_ohe.reshape(-1, 467, 1)


In [91]:
# printing new 3 dimensional shapes of our one hot encodeded data
print(x_train_ohe.shape)
print(x_cv_ohe.shape)
print(x_test_ohe.shape)

(3200, 467, 1)
(800, 467, 1)
(1000, 467, 1)


In [100]:
ip_seq_total_data1 = Input(shape=(400,), name='total_data')
x1 = Embedding(input_dim=400000, output_dim=128, input_length=400)(ip_seq_total_data1)
x1 = LSTM(32, return_sequences=True)(x1)
x1 = Flatten()(x1)

other_than_text_data = Input(shape=(467, 1), name='no_text_data')
y1 = Conv1D(filters=32, kernel_size=4, activation='relu')(other_than_text_data)
y1 = Conv1D(filters=16, kernel_size=8, activation='relu')(y1)
y1 = Flatten()(y1)

model = concatenate([x1, y1])

model = Dense(64, activation='relu')(model)
model = Dropout(0.5)(model)
model = Dense(64, activation='relu')(model)
model = Dropout(0.25)(model)
model = Dense(32, activation='relu')(model)
output_layer = Dense(2, activation='softmax')(model)


model_3 = Model(inputs = [ip_seq_total_data1,other_than_text_data], outputs= output_layer)
# Compiling the model
model_3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model_3.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
total_data (InputLayer)         (None, 400)          0                                            
__________________________________________________________________________________________________
no_text_data (InputLayer)       (None, 467, 1)       0                                            
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 400, 128)     51200000    total_data[0][0]                 
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 464, 32)      160         no_text_data[0][0]               
__________________________________________________________________________________________________
lstm_22 (L

In [101]:
model_3.fit([padded_train_essay, x_train_ohe],y_train, epochs=10, batch_size=128, verbose=2, validation_data=([padded_cv_essay, x_cv_ohe], y_cv))

Train on 3200 samples, validate on 800 samples
Epoch 1/10
 - 44s - loss: 0.4947 - acc: 0.8191 - val_loss: 0.4090 - val_acc: 0.8575
Epoch 2/10
 - 38s - loss: 0.4150 - acc: 0.8478 - val_loss: 0.4074 - val_acc: 0.8575
Epoch 3/10
 - 38s - loss: 0.3014 - acc: 0.8528 - val_loss: 0.5439 - val_acc: 0.8550
Epoch 4/10
 - 38s - loss: 0.1241 - acc: 0.9528 - val_loss: 0.6736 - val_acc: 0.8263
Epoch 5/10
 - 37s - loss: 0.0458 - acc: 0.9866 - val_loss: 0.9496 - val_acc: 0.7850
Epoch 6/10
 - 37s - loss: 0.0088 - acc: 0.9988 - val_loss: 1.5104 - val_acc: 0.8313
Epoch 7/10
 - 37s - loss: 0.0038 - acc: 0.9988 - val_loss: 1.6792 - val_acc: 0.8125
Epoch 8/10
 - 37s - loss: 0.0018 - acc: 1.0000 - val_loss: 1.8458 - val_acc: 0.8413
Epoch 9/10
 - 38s - loss: 0.0081 - acc: 0.9984 - val_loss: 1.6535 - val_acc: 0.8187
Epoch 10/10
 - 38s - loss: 0.0036 - acc: 0.9988 - val_loss: 1.5266 - val_acc: 0.8037
